In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:0

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
!mkdir pdfs

In [ ]:
!gdown 1ya0aP1AkHymDRTtbR8gzL6s9lFyuxss5 -O pdfs/n12.pdf

Downloading...
From: https://drive.google.com/uc?id=1ya0aP1AkHymDRTtbR8gzL6s9lFyuxss5
To: /content/pdfs/n12.pdf
100% 1.15M/1.15M [00:00<00:00, 10.3MB/s]


In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='1E-BUSINESS\nPROF. MAMATA JENAMANI\nDEPARTMENT OF INDUSTRIAL AND SYSTEMS ENGINEERING\nIIT KHARAGPUR', metadata={'source': 'pdfs/n12.pdf', 'page': 0}), Document(page_content='ECONOMIC CONSIDERAIONS IN AUCTIONWeek 12: Lecture1', metadata={'source': 'pdfs/n12.pdf', 'page': 1}), Document(page_content='We are going to learn\n•Auction design problem\n•Efficiency and optimality considerations\n•Example of auction mechanism design\n3', metadata={'source': 'pdfs/n12.pdf', 'page': 2}), Document(page_content='Auction Design Problem\n•Deals with economic considerations\n•Designing auctions rules with some desirable \nproperty to satisfy the auctioneers need\n–Modeling preference, behavior and information \navailable to the agents\n–Designing mechanisms in which the agent strategies \nresult in the outcomes with the desirable properties.', metadata={'source': 'pdfs/n12.pdf', 'page': 3}), Document(page_content='Modeling bidders valuation of a product\n•Private value model\n–E

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
docs = text_splitter.split_documents(data)

In [ ]:
print(len(docs))

55


In [ ]:
docs[3]

Document(page_content='Auction Design Problem\n•Deals with economic considerations\n•Designing auctions rules with some desirable \nproperty to satisfy the auctioneers need\n–Modeling preference, behavior and information \navailable to the agents\n–Designing mechanisms in which the agent strategies \nresult in the outcomes with the desirable properties.', metadata={'source': 'pdfs/n12.pdf', 'page': 3})

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query_result = embeddings.embed_query("Hello")

In [ ]:
print("Length", len(query_result))

Length 384


In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "heliopine" # put in the name of your pinecone index here


In [ ]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [ ]:
query = "Nature of business on the basis of organization"

In [ ]:
docs = docsearch.similarity_search(query, k=4)

In [ ]:
docs

[Document(page_content='The accompanying notes are an integral part of these unaudited condensed financial statements.  \n6 \nTable of Contents  \nIMMUNOME,  INC.  \nNotes  to Condensed Financial Statements  \n(Unaudited)  \n1. Nature of the business  \nOrganization  \nImmunome, Inc., the Company or Immunome, is a biopharmaceutical company. The Company was incorporated as a Pennsylvania corpo ration on March 2, 2006 and was'),
 Document(page_content='achievement of significant regulatory and development events, commercial sales milestones, and royalties on product sales. Th e amount of variable considera tion is constrained until it \nis probable that the revenue is not at a significant risk of reversal in a future period.  \nIn determining the appropriate amount of revenue to be recognized as the Company fulfills its obligations under a collaborati on arrangeme nt, the Company applies the five -step'),
 Document(page_content='● the achievement of milestones or occurrence of other deve

In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
pip install transformers

In [ ]:
import transformers
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
template

"[INST]<>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
# result = qa_chain("what is the nature of business on the basis of organization")

In [ ]:
# result['result']

"  Based on the information provided in the notes to the condensed financial statements, the nature of Immunome, Inc.'s business is that of a biopharmaceutical company."

In [ ]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:a
